# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
#Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

#Import API key
from api_keys import geoapify_key

In [2]:
#Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Resources/cities.csv")

#Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,57.81,91,97,7.00,NZ,Thu Oct 26 16:32:53 2023
1,1,qaqortoq,60.7167,-46.0333,32.86,63,31,3.83,GL,Thu Oct 26 16:29:16 2023
2,2,hithadhoo,-0.6000,73.0833,81.25,77,33,16.89,MV,Thu Oct 26 16:32:55 2023
3,3,yellowknife,62.4560,-114.3525,30.42,77,75,16.11,CA,Thu Oct 26 16:31:03 2023
4,4,edinburgh of the seven seas,-37.0676,-12.3116,54.28,62,100,9.86,SH,Thu Oct 26 16:32:57 2023


In [3]:
null_columns = city_data_df.columns[city_data_df.isnull().any()]
city_data_df[null_columns].isnull().sum()

Country    4
dtype: int64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

#Configure the map plot
map_humidity = city_data_df.hvplot.points('Lng', 'Lat', geo=True, scale=1, color='City', size='Humidity', frame_width=700, frame_height=500, alpha=0.5, tiles='OSM')

#Display the map
map_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
#Narrow down cities that fit criteria
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] >= 15)
                                & (city_data_df['Max Temp'] <= 25)
                                # & (city_data_df['Cloudiness'] == 0)
                                # & (city_data_df['Wind Speed'] <= 2)
]

#Drop any rows with null values
ideal_weather_df.dropna()

#Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,70,whitehorse,60.7161,-135.0538,21.97,86,40,10.36,CA,Thu Oct 26 16:33:55 2023
134,134,leninogorsk,54.6026,52.4609,23.85,92,9,2.73,RU,Thu Oct 26 16:34:52 2023
150,150,vuktyl,63.8567,57.3094,20.10,88,100,5.01,RU,Thu Oct 26 16:35:05 2023
196,196,aasiaat,68.7098,-52.8699,24.75,91,98,8.41,GL,Thu Oct 26 16:35:49 2023
210,210,nyurba,63.2842,118.3319,18.63,97,100,3.09,RU,Thu Oct 26 16:36:02 2023
244,244,casper,42.8666,-106.3131,21.72,88,100,4.00,US,Thu Oct 26 16:34:56 2023
254,254,magadan,59.5667,150.8000,20.61,79,70,5.32,RU,Thu Oct 26 16:36:45 2023
263,263,ola,59.5833,151.2833,24.42,65,59,6.15,RU,Thu Oct 26 16:36:52 2023
279,279,longyearbyen,78.2186,15.6401,24.64,80,75,10.36,SJ,Thu Oct 26 16:37:05 2023
289,289,qamdo,31.1667,97.2333,24.78,75,74,3.51,CN,Thu Oct 26 16:37:15 2023


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
#Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

#Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

#Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
70,whitehorse,CA,60.7161,-135.0538,86,
134,leninogorsk,RU,54.6026,52.4609,92,
150,vuktyl,RU,63.8567,57.3094,88,
196,aasiaat,GL,68.7098,-52.8699,91,
210,nyurba,RU,63.2842,118.3319,97,
244,casper,US,42.8666,-106.3131,88,
254,magadan,RU,59.5667,150.8000,79,
263,ola,RU,59.5833,151.2833,65,
279,longyearbyen,SJ,78.2186,15.6401,80,
289,qamdo,CN,31.1667,97.2333,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
#Set parameters to search for a hotel
radius = 10000

params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key
}

#Print a message to follow up the hotel search
print('Starting hotel search')

#Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    #Get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    #Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{longitude}, {latitude}, {radius}'
    params['bias'] = f'proximity:{longitude}, {latitude}'
    
    #Set base URL
    base_url = 'https://api.geoapify.com/v2/places'


    #Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    #Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, 'Hotel Name'] = 'No hotel found'
        
    #Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

#Display sample data
hotel_df

Starting hotel search
whitehorse - nearest hotel: No hotel found
leninogorsk - nearest hotel: No hotel found
vuktyl - nearest hotel: No hotel found
aasiaat - nearest hotel: No hotel found
nyurba - nearest hotel: No hotel found
casper - nearest hotel: No hotel found
magadan - nearest hotel: No hotel found
ola - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
qamdo - nearest hotel: No hotel found
skovorodino - nearest hotel: No hotel found
turinsk - nearest hotel: No hotel found
slave lake - nearest hotel: No hotel found
zaozersk - nearest hotel: No hotel found
okhotsk - nearest hotel: No hotel found
mezhdurechenskiy - nearest hotel: No hotel found
lockwood - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
70,whitehorse,CA,60.7161,-135.0538,86,No hotel found
134,leninogorsk,RU,54.6026,52.4609,92,No hotel found
150,vuktyl,RU,63.8567,57.3094,88,No hotel found
196,aasiaat,GL,68.7098,-52.8699,91,No hotel found
210,nyurba,RU,63.2842,118.3319,97,No hotel found
244,casper,US,42.8666,-106.3131,88,No hotel found
254,magadan,RU,59.5667,150.8000,79,No hotel found
263,ola,RU,59.5833,151.2833,65,No hotel found
279,longyearbyen,SJ,78.2186,15.6401,80,No hotel found
289,qamdo,CN,31.1667,97.2333,75,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

#Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng', 
                                    'Lat', 
                                    geo=True, 
                                    tiles='OSM', 
                                    color='City', 
                                    size='Humidity', 
                                    hover_cols=['Hotel Name', 'Country'])

#Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)